In [1]:
from sklearn.linear_model import RANSACRegressor


In [12]:
import numpy as np
import open3d as o3d

# 파일 경로 설정
file_path = "C:\\Users\\rjhyu\\파이썬 연습\\2024 new\\텍스트 파일\\230316C_15mm0deg-20개.txt"
# .txt 파일을 NumPy 배열로 로드
# delimiter는 파일에 따라 조정할 수 있습니다 (공백, 탭, 쉼표 등)
data = np.loadtxt(file_path, delimiter=' ')  # 공백으로 구분된 경우

# Open3D 포인트 클라우드 객체 생성
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(data[:, :3])  # x, y, z 좌표 사용

# 레이저 스캔 데이터에서 노이즈를 줄이기 위한 사전 처리
# 예: 다운샘플링, 이상치 제거 등
pcd_down = pcd.voxel_down_sample(voxel_size=0.02)  # 다운샘플링을 위한 예시, voxel_size는 데이터에 따라 조정

# RANSAC을 사용하여 첫 번째 평면 검출
plane1_model, inliers1 = pcd_down.segment_plane(distance_threshold=0.01, ransac_n=3, num_iterations=1000)
inlier_cloud1 = pcd_down.select_by_index(inliers1)

# 첫 번째 평면에 속하지 않는 데이터 필터링
outlier_cloud = pcd_down.select_by_index(inliers1, invert=True)

# RANSAC을 사용하여 두 번째 평면 검출
plane2_model, inliers2 = outlier_cloud.segment_plane(distance_threshold=0.01, ransac_n=3, num_iterations=1000)
inlier_cloud2 = outlier_cloud.select_by_index(inliers2)

# 첫 번째 및 두 번째 평면의 평균 높이(가정: z축이 높이) 계산
avg_height1 = np.mean(np.asarray(inlier_cloud1.points)[:,2])
avg_height2 = np.mean(np.asarray(inlier_cloud2.points)[:,2])

# 두 평면의 높이 차이 계산
height_difference = abs(avg_height1 - avg_height2)

# 높이 차이가 작다면, 두 평면은 동일한 높이에 있다고 간주
if height_difference < 0.05:  # 0.05는 임의의 임계값이며, 필요에 따라 조정
    print("두 평면은 같거나 거의 동일한 높이에 있습니다.")
else:
    print("두 평면은 다른 높이에 있습니다.")

# 결과 시각화
inlier_cloud1.paint_uniform_color([1.0, 0, 0])  # 첫 번째 평면은 빨간색
inlier_cloud2.paint_uniform_color([0, 1.0, 0])  # 두 번째 평면은 초록색
o3d.visualization.draw_geometries([inlier_cloud1, inlier_cloud2])


두 평면은 같거나 거의 동일한 높이에 있습니다.


In [10]:
import numpy as np
import open3d as o3d

# 파일 경로 설정
file_path = "C:\\Users\\rjhyu\\파이썬 연습\\2024 new\\텍스트 파일\\230316C_15mm0deg-20개.txt"

# .txt 파일을 NumPy 배열로 로드
data = np.loadtxt(file_path, delimiter=' ')  # 공백으로 구분된 경우

# Open3D 포인트 클라우드 객체 생성
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(data[:, :3])  # x, y, z 좌표 사용

# 관심 있는 영역(ROI) 정의 (예: x, y, z 좌표의 특정 범위)
x_min, x_max = -1, 1  # 이 값을 실제 관심 영역에 맞게 조정하세요.
y_min, y_max = -1, 1  # 이 값을 실제 관심 영역에 맞게 조정하세요.
z_min, z_max = -1, 1  # 이 값을 실제 관심 영역에 맞게 조정하세요.

# ROI에 해당하는 데이터만 필터링
points = np.asarray(pcd.points)
roi_indices = np.where(
    (points[:, 0] >= x_min) & (points[:, 0] <= x_max) & 
    (points[:, 1] >= y_min) & (points[:, 1] <= y_max) & 
    (points[:, 2] >= z_min) & (points[:, 2] <= z_max)
)[0]
roi_pcd = pcd.select_by_index(roi_indices)

# 필터링된 데이터에 대해 다운샘플링 적용
down_pcd = roi_pcd.voxel_down_sample(voxel_size=0.02)  # 다운샘플링을 위한 예시

# 다운샘플링된 데이터와 원본 데이터의 나머지 부분을 병합
remaining_pcd = pcd.select_by_index(roi_indices, invert=True)
merged_pcd = down_pcd + remaining_pcd

# 결과 시각화
o3d.visualization.draw_geometries([merged_pcd])


In [16]:
import numpy as np
import open3d as o3d

# 파일 로드 및 포인트 클라우드 생성
file_path = "C:\\Users\\rjhyu\\파이썬 연습\\for txt file\\row data\\23.03.17 Cross 다시 측정\\23.03.17 Cross 다시 측정\\C_15mm\\15mm0deg\\230316C_15mm0deg-20개.txt"  # 파일 경로 설정
data = np.loadtxt(file_path, delimiter=' ')  # 공백으로 구분된 경우
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(data[:, :3])  # x, y, z 좌표 사용

# y 값이 0.049부터 0.051 사이에 있는 데이터 선택
indices = np.where((np.asarray(pcd.points)[:, 1] >= 0.049) & (np.asarray(pcd.points)[:, 1] <= 0.051))[0]
roi_pcd = pcd.select_by_index(indices)

# 노이즈 제거 (예: 다운샘플링)
roi_pcd_down = roi_pcd.voxel_down_sample(voxel_size=0.02)

# 노이즈가 제거된 포인트 클라우드와 나머지 데이터 결합
rest_pcd = pcd.select_by_index(indices, invert=True)
combined_pcd = rest_pcd + roi_pcd_down

# 결과 시각화
o3d.visualization.draw_geometries([combined_pcd])


In [4]:
import pandas as pd
import open3d as o3d
import numpy as np

# Define the path to your .txt file
file_path = 'C:/Users/rjhyu/파이썬 연습/23.12.09 오류 원인 찾기/C15mm00deg.txt'

# Read the file and extract the x, y, z values
data = []
with open(file_path, 'r') as file:
    for line in file:
        values = line.strip().split()  # Split the line into individual values
        if len(values) >= 3:
            try:
                x, y, z = float(values[0]), float(values[1]), float(values[2])  # Extract x, y, z
                data.append([x, y, z])
            except ValueError:
                # Skip lines that cannot be converted to floats
                continue

# Create a DataFrame from the extracted data
df = pd.DataFrame(data, columns=['x', 'y', 'z'])

# Convert DataFrame to Open3D PointCloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(df[['x', 'y', 'z']].values)

# Apply RANSAC to detect the two planes
distance_threshold = 0.01
ransac_n = 3
num_iterations = 1000

# Detect the first plane
plane1_model, inliers1 = pcd.segment_plane(distance_threshold, ransac_n, num_iterations)
inlier_cloud1 = pcd.select_by_index(inliers1)

# Detect the second plane
inlier_cloud1.paint_uniform_color([1, 0, 0])  # Color the first plane in red
outlier_cloud = pcd.select_by_index(inliers1, invert=True)
plane2_model, inliers2 = outlier_cloud.segment_plane(distance_threshold, ransac_n, num_iterations)
inlier_cloud2 = outlier_cloud.select_by_index(inliers2)
inlier_cloud2.paint_uniform_color([0, 1, 0])  # Color the second plane in green

# Calculate the distance between the two planes
avg_height1 = np.mean(np.asarray(inlier_cloud1.points)[:, 2])
avg_height2 = np.mean(np.asarray(inlier_cloud2.points)[:, 2])
distance_between_planes = abs(avg_height1 - avg_height2)

print(f"Distance between the two planes: {distance_between_planes} units")

# Visualize the two planes
o3d.visualization.draw_geometries([inlier_cloud1, inlier_cloud2])


ModuleNotFoundError: No module named 'pandas'